In [6]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
import sys

sys.path.append("/workspace/kbqa/")  # go to parent dir

In [8]:
import ujson
import jsonlines
import networkx as nx
import pandas as pd
from tqdm import tqdm
from pathlib import Path

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
import torch
import torch.nn.functional as F

2023-07-26 16:44:37.554549: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 16:44:37.794545: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-26 16:44:38.625625: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-07-26 16:44:38.625704: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

#### Getting JSONL subgraphs dataset

In [9]:
dataset_type = 't5-xl-ssm'
new_test_dataset = False 
train_bs = 16
eval_bs = 32
is_special_tok_context = True
model_weights = f"/workspace/storage/subgraphs_reranking_results/{dataset_type}/results/mse_subgraph_mpnet_ranking_T5XLSSMNQ"
model_name = "roberta-large"
model_save_name = f"{model_name}_mse_token_context" if not model_weights else model_weights.split('/')[-1]

In [10]:
if model_weights: # evaluating
    tokenizer = AutoTokenizer.from_pretrained(model_weights)
    model = AutoModelForSequenceClassification.from_pretrained(model_weights).to(
        device
    )
else: # training from scratch
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.add_special_tokens(
        {"additional_special_tokens": ["[unused1]", "[unused2]"]}
    )
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1).to(
        device
    )

In [11]:
dataset_type = 't5-large-ssm'
new_test_dataset = False 
train_bs = 16
eval_bs = 32
is_special_tok_context = True
model_weights = f"/workspace/storage/subgraphs_reranking_results/{dataset_type}/results/mse_subgraph_bert_ranking_T5LargeSSM"
model_name = "roberta-large"
model_save_name = f"{model_name}_mse_token_context" if not model_weights else model_weights.split('/')[-1]

In [5]:
if model_weights: # evaluating
    tokenizer = AutoTokenizer.from_pretrained(model_weights)
    model = AutoModelForSequenceClassification.from_pretrained(model_weights).to(
        device
    )
else: # training from scratch
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.add_special_tokens(
        {"additional_special_tokens": ["[unused1]", "[unused2]"]}
    )
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1).to(
        device
    )

In [6]:
def read_jsonl(path):
    jsonl_reader = jsonlines.open(path)
    jsonl_reader_list = list(jsonl_reader)
    df = []
    for line in tqdm(jsonl_reader_list):
        df.append(line)
    df = pd.DataFrame(df)
    return df


train_df = read_jsonl(
    f"/workspace/storage/new_subgraph_dataset/{dataset_type}/mintaka_train_labeled.jsonl"
)
val_df = read_jsonl(
    f"/workspace/storage/new_subgraph_dataset/{dataset_type}/mintaka_validation_labeled.jsonl"
)
test_df = read_jsonl(
    f"/workspace/storage/new_subgraph_dataset/{dataset_type}/mintaka_test_labeled.jsonl"
)

100%|██████████| 27094/27094 [00:00<00:00, 676113.45it/s]


#### Converting graph to its sequences

In [12]:
def get_node_names(subgraph, candidate_start_token="[unused1]", candidate_end_token="[unused2]",):
    node_names = [subgraph.nodes[node]["label"] for node in subgraph.nodes()]
    node_type = [subgraph.nodes[node]["type"] for node in subgraph.nodes()]
    
    if 'ANSWER_CANDIDATE_ENTITY' not in node_type:
        return None

    if is_special_tok_context:
        candidate_idx = node_type.index("ANSWER_CANDIDATE_ENTITY")
        node_names[
            candidate_idx
        ] = f"{candidate_start_token}{node_names[candidate_idx]}{candidate_end_token}"
    
    return node_names

In [13]:
def graph_to_sequence(subgraph, node_names):
    # getting adjency matrix and weight info
    adj_matrix = nx.adjacency_matrix(subgraph).todense().tolist()
    edge_data = subgraph.edges.data()

    # adding our edge info
    for edge in edge_data:
        i, j, data = edge
        i, j = int(i), int(j)
        adj_matrix[i][j] = data["label"]

    sequence = []
    # for adjency matrix, i, j means node i -> j
    for i, row in enumerate(adj_matrix):
        from_node = node_names[i]  # from node (node i)
        for j, edge_info in enumerate(row):
            to_node = node_names[j]
            if edge_info != 0:  # no endge from_node -> to_node
                sequence.extend([from_node, edge_info, to_node])
    
    sequence = ",".join(str(node) for node in sequence)
    return sequence

In [14]:
from ast import literal_eval
from unidecode import unidecode


def try_literal_eval(s):
    try:
        return literal_eval(s)
    except ValueError:
        return s


def get_sequences(df):
    questions = list(df["question"])
    graphs = list(df["graph"])
    graph_seq = []
    for question, graph in tqdm(zip(questions, graphs)):
        graph_obj = nx.readwrite.json_graph.node_link_graph(try_literal_eval(graph))
        try:
            graph_node_names = get_node_names(graph_obj)
            if graph_node_names is None:
                curr_seq = "ERROR_NO_CANDIDATE"
            else:     
                curr_seq = graph_to_sequence(graph_obj, graph_node_names)
                if is_special_tok_context:
                    curr_seq = f"{question}{tokenizer.sep_token}{curr_seq}"
        except KeyError:
            curr_seq = "ERROR_NO_LABEL"
        except nx.NetworkXError:
            curr_seq = "ERROR_EMPTY_GRAPH"
        graph_seq.append(curr_seq)
    
    return graph_seq

In [15]:
def preprocess_data(df):
    # get the sequences
    df_sequences = get_sequences(df)
    df["graph_sequence"] = df_sequences
    
    # filter out all invalid graphs
    error_df = df[
        (df["graph_sequence"] == "ERROR_EMPTY_GRAPH")
        | (df["graph_sequence"] == "ERROR_NO_LABEL")
        | (df["graph_sequence"] == "ERROR_NO_CANDIDATE")
    ]
    df = df.drop(error_df.index)
    
    # turn list of entities into string
    df["answerEntity"] = df["answerEntity"].apply(lambda x: ", ".join(x))
    df["questionEntity"] = df["questionEntity"].apply(lambda x: ", ".join(x))
    df["groundTruthAnswerEntity"] = df["groundTruthAnswerEntity"].apply(
        lambda x: ", ".join(x)
    )
    df["correct"] = df.apply(
        lambda x: x["answerEntity"] in x["groundTruthAnswerEntity"], axis=1
    )
    
    return df

In [16]:
# get train and test texts & labels
concat_train_df = pd.concat([train_df, val_df])

In [17]:
concat_train_df = preprocess_data(concat_train_df)
concat_train_df.head()

108336it [00:27, 3937.33it/s]


id                                           question answerEntity  \
0  2723bb1b  Which actor was the star of Titanic and was bo...   Q100711983   
1  a9011ddf  What is the seventh tallest mountain in North ...      Q130018   
2  982450cf           Who is the youngest current US governor?       Q11673   
3  982450cf           Who is the youngest current US governor?          Q30   
4  982450cf           Who is the youngest current US governor?      Q132050   

  questionEntity groundTruthAnswerEntity complexityType  \
0    Q44578, Q65                  Q38111   intersection   
1            Q49                Q1153188        ordinal   
2        Q889821                Q3105215    superlative   
3        Q889821                Q3105215    superlative   
4        Q889821                Q3105215    superlative   

                                               graph  \
0  {'directed': True, 'multigraph': False, 'graph...   
1  {'directed': True, 'multigraph': False, 'graph...   
2  {'directed': True, 'multigraph': False, 'graph...   
3  {'directed': True, 'multigraph': False, 'graph...   
4  {'directed': True, 'multigraph': False, 'graph...   

                                      graph_sequence  correct  
0  Which actor was the star of Titanic and was bo...    False  
1  What is the seventh tallest mountain in North ...    False  
2  Who is the youngest current US governor?</s>Un...    False  
3  Who is the youngest current US governor?</s>[u...    False  
4  Who is the youngest current US governor?</s>go...    False

In [18]:
test_df = preprocess_data(test_df)
test_df.head()

0it [00:00, ?it/s]

27094it [00:06, 4114.62it/s]


,id,question,answerEntity,questionEntity,groundTruthAnswerEntity,complexityType,graph,graph_sequence,correct
0,fae46b21,What man was a famous American author and also...,Q191050,"Q1497, Q846570",Q7245,intersection,"{'directed': True, 'multigraph': False, 'graph...",What man was a famous American author and also...,False
1,fae46b21,What man was a famous American author and also...,Q3259878,"Q1497, Q846570",Q7245,intersection,"{'directed': True, 'multigraph': False, 'graph...",What man was a famous American author and also...,False
2,fae46b21,What man was a famous American author and also...,Q7245,"Q1497, Q846570",Q7245,intersection,"{'directed': True, 'multigraph': False, 'graph...",What man was a famous American author and also...,True
3,fae46b21,What man was a famous American author and also...,Q1074614,"Q1497, Q846570",Q7245,intersection,"{'directed': True, 'multigraph': False, 'graph...",What man was a famous American author and also...,False
4,fae46b21,What man was a famous American author and also...,Q15133865,"Q1497, Q846570",Q7245,intersection,"{'directed': True, 'multigraph': False, 'graph...",What man was a famous American author and also...,False


#### Building our dataset

In [19]:
train_texts = concat_train_df["graph_sequence"].tolist()
train_labels = concat_train_df["correct"].astype(int).tolist()  # convert true false to 1 0

test_texts = test_df["graph_sequence"].tolist()
test_labels = test_df["correct"].astype(int).tolist()  # convert true false to 1 0

In [20]:
# get our encodings
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [21]:
class SequenceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = SequenceDataset(train_encodings, train_labels)
test_dataset = SequenceDataset(test_encodings, test_labels)

#### Training

In [22]:
import numpy as np
import evaluate

threshold = 0.5
metric_classifier = evaluate.combine(["accuracy", "f1", "precision", "recall", "hyperml/balanced_accuracy",])
metric_regression = evaluate.combine(["mae"])


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    results = metric_regression.compute(predictions=predictions, references=labels)

    predictions = predictions > threshold
    results.update(
        metric_classifier.compute(predictions=predictions, references=labels)
    )

    return results

In [23]:
# Specifiy the arguments for the trainer
training_args = TrainingArguments(
    output_dir=f"/workspace/storage/subgraphs_reranking_results/t5-xl-ssm/results/{model_save_name}",  # output directory
    num_train_epochs=5,  # total number of training epochs
    per_device_train_batch_size=train_bs,  # batch size per device during training
    per_device_eval_batch_size=eval_bs,  # batch size for evaluation
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    logging_dir=f"/workspace/storage/subgraphs_reranking_results/t5-xl-ssm/logs/{model_save_name}",  # directory for storing logs
    load_best_model_at_end=True,  # load the best model when finished training (default metric is loss)
    metric_for_best_model="balanced_accuracy",  # select the base metrics
    logging_steps=500,  # log & save weights each logging_steps
    save_steps=500,
    evaluation_strategy="steps",  # evaluate each `logging_steps`
    #report_to='wandb',
)

In [24]:
from torch.utils.data.sampler import WeightedRandomSampler
import numpy as np

def create_sampler(target):
    class_sample_count = np.array(
        [len(np.where(target == t)[0]) for t in np.unique(target)]
    )
    weight = 1.0 / class_sample_count
    samples_weight = np.array([weight[t] for t in target])

    samples_weight = torch.from_numpy(samples_weight)
    samples_weigth = samples_weight.double()
    sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

    return sampler

In [25]:
class CustomTrainer(Trainer):    
    def get_train_dataloader(self) -> torch.utils.data.DataLoader:
        train_sampler = create_sampler(concat_train_df["correct"].astype(int).ravel())
        train_loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=train_bs, sampler=train_sampler
        )
        return train_loader

In [26]:
# Call the Trainer
trainer = CustomTrainer(
    model=model,  # the instantiated Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=train_dataset,  # training dataset
    eval_dataset=test_dataset,  # evaluation dataset
    compute_metrics=compute_metrics,  # the callback that computes metrics of interest
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [27]:
# Train the model
if not model_weights: # training
    trainer.train()

#### Evaluating

In [28]:
evaluate_res = trainer.evaluate()
evaluate_res

***** Running Evaluation *****
  Num examples = 26759
  Batch size = 32


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: hle2000. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'eval_loss': 0.13944019377231598,
 'eval_mae': 0.21075966312410827,
 'eval_accuracy': 0.8092230651369633,
 'eval_f1': 0.5954513035898249,
 'eval_precision': 0.4442999053926206,
 'eval_recall': 0.9024741772760029,
 'eval_balanced_accuracy': 0.847258508358306,
 'eval_runtime': 220.7764,
 'eval_samples_per_second': 121.204,
 'eval_steps_per_second': 3.791}

#### Final Re-ranking

In [42]:
if not new_test_dataset:
    res_csv = pd.read_csv(
        f"/workspace/storage/mintaka_seq2seq/{dataset_type}/test/results.csv"
    )
    final_acc, top200_total, top1_total, seq2seq_correct = 0, 0, 0, 0
    
    for idx, group in tqdm(res_csv.iterrows()):
        curr_question_df = test_df[test_df["question"] == group['question']]
        if len(curr_question_df) == 0: # we don't have subgraph for this question, take answer from seq2seq
            if group["answer_0"] == group["target"]:
                seq2seq_correct += 1
            else: # check if answer exist in 200 beams for question with no subgraphs
                all_beams = group.tolist()[2:-1] # all 200 beams
                all_beams = list(set(all_beams))
                top200_total += 1 if group["target"] in all_beams else 0
                
        else: # we have subgraph for this question
            all_beams = group.tolist()[2:-1] # all 200 beams
            all_beams = list(set(all_beams))
            
            if group["target"] not in all_beams: # no correct answer in beam
                continue
            
            # correct answer exist in beam
            top1_total += 1 if group["answer_0"] == group["target"] else 0
            top200_total += 1
            
            # reranking
            seqs = curr_question_df["graph_sequence"].tolist()
            is_corrects = curr_question_df["correct"].tolist()
            
            tok_seq = tokenizer(seqs, padding="max_length",
                                max_length=512,
                                truncation=True,
                                return_tensors="pt",
                                )
            mask = tok_seq["attention_mask"].to(device)
            input_id = tok_seq["input_ids"].squeeze(1).to(device)
            output = model(input_id, mask).logits
            output = torch.flatten(output)

            max_idx = output.argmax(dim=0).item()

            if is_corrects[max_idx] is True:
                final_acc += 1
    
    # final rerankinga, top1 and top200 result
    reranking_res = (final_acc + seq2seq_correct)/ len(res_csv)
    top200 = (top200_total + seq2seq_correct)/len(res_csv)
    top1 = (top1_total + seq2seq_correct)/ len(res_csv)


4000it [02:56, 22.63it/s] 


In [31]:
if new_test_dataset: # new_test_dataset: # reranking for new test dataset format
    test_dataset_path = '/workspace/storage/new_subgraph_dataset/t5-xl-ssm/mintaka_test_labeled_new.jsonl'
    new_test_df = read_jsonl(test_dataset_path)
    new_test_df = preprocess_data(new_test_df)
    new_test_df_group = new_test_df.groupby('question')
    final_acc, top200_total = 0, 0

    for question, row in new_test_df_group:
        answers = row['answerEntity'].tolist()
        ground_truth = row['groundTruthAnswerEntity'].tolist()[0]
        if ground_truth in answers:
            top200_total += 1
        # reranking
        seqs = row["graph_sequence"].tolist()
        is_corrects = row["correct"].tolist()
        
        tok_seq = tokenizer(seqs, padding="max_length",
                            max_length=512,
                            truncation=True,
                            return_tensors="pt",
                            )
        mask = tok_seq["attention_mask"].to(device)
        input_id = tok_seq["input_ids"].squeeze(1).to(device)
        output = model(input_id, mask).logits
        output = torch.flatten(output)

        max_idx = output.argmax(dim=0).item()

        if is_corrects[max_idx] is True:
            final_acc += 1   
    
    # final rerankinga, top1 and top200 result
    reranking_res = final_acc / len(new_test_df_group)
    top200 = top200_total/len(new_test_df_group)
    top1 = 'Not available for new test dataset'

In [32]:
# saving the final result to txt file
with open(
    f"/workspace/storage/subgraphs_reranking_results/{dataset_type}/results/{model_save_name}/final_results_seq2seq.txt",
    "w+",
) as f:
    f.write(f'original top1: {top1}, top200: {top200}\n')
    f.write(f"Final reranking accuracy: {reranking_res}\n")
    f.write("\n")
    f.write("trainer.evaluate() result:\n")
    for k, v in evaluate_res.items():
        f.write(f"{k}:{v}\n")